In [2]:
from os import environ

environ['CUDA_VISIBLE_DEVICES'] = '-1'

from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import Adam
from numpy import load
from onnx import save
from tf2onnx import convert

In [4]:
print('training model')

epoch_count = int(environ.get('epoch_count', '20'))
learning_rate = float(environ.get('learning_rate', '0.001'))

Xsm_train = load(f'data/training_samples.npy')
ysm_train = load(f'data/training_labels.npy')
n_inputs = Xsm_train.shape[1]

model = Sequential([
    Dense(n_inputs, input_shape=(n_inputs, ), activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax'),
])
model.compile(
    Adam(learning_rate=learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)
model.fit(
    Xsm_train,
    ysm_train,
    validation_split=0.2,
    batch_size=300,
    epochs=epoch_count,
    shuffle=True,
    verbose=2,
)

training model
Epoch 1/20
1214/1214 - 4s - loss: 0.0819 - accuracy: 0.9674 - val_loss: 0.0390 - val_accuracy: 0.9864 - 4s/epoch - 3ms/step
Epoch 2/20
1214/1214 - 3s - loss: 0.0169 - accuracy: 0.9956 - val_loss: 0.0110 - val_accuracy: 0.9996 - 3s/epoch - 2ms/step
Epoch 3/20
1214/1214 - 3s - loss: 0.0084 - accuracy: 0.9983 - val_loss: 0.0065 - val_accuracy: 0.9999 - 3s/epoch - 2ms/step
Epoch 4/20
1214/1214 - 3s - loss: 0.0057 - accuracy: 0.9989 - val_loss: 0.0025 - val_accuracy: 1.0000 - 3s/epoch - 2ms/step
Epoch 5/20
1214/1214 - 3s - loss: 0.0043 - accuracy: 0.9991 - val_loss: 0.0029 - val_accuracy: 1.0000 - 3s/epoch - 2ms/step
Epoch 6/20
1214/1214 - 3s - loss: 0.0035 - accuracy: 0.9993 - val_loss: 0.0016 - val_accuracy: 1.0000 - 3s/epoch - 2ms/step
Epoch 7/20
1214/1214 - 3s - loss: 0.0028 - accuracy: 0.9994 - val_loss: 0.0019 - val_accuracy: 1.0000 - 3s/epoch - 2ms/step
Epoch 8/20
1214/1214 - 3s - loss: 0.0025 - accuracy: 0.9995 - val_loss: 0.0017 - val_accuracy: 0.9999 - 3s/epoch - 2m

# Export the model to ONNX

In [6]:
model, _ = convert.from_keras(model)
save(model, 'model.onnx')